https://stackoverflow.com/questions/70537488/cannot-import-name-registermattype-from-cv2-cv2

In [ ]:
!pip install opencv-python-headless==4.1.2.30

     |████████████████████████████████| 21.8 MB 1.5 MB/s 
  Attempting uninstall: opencv-python-headless
    Found existing installation: opencv-python-headless 4.5.5.62
    Uninstalling opencv-python-headless-4.5.5.62:
      Successfully uninstalled opencv-python-headless-4.5.5.62


In [ ]:
!pip install -U albumentations

     |████████████████████████████████| 102 kB 4.5 MB/s 
     |████████████████████████████████| 47.7 MB 2.0 MB/s 
  Attempting uninstall: albumentations
    Found existing installation: albumentations 0.1.12
    Uninstalling albumentations-0.1.12:
      Successfully uninstalled albumentations-0.1.12


In [ ]:
import cv2
import albumentations as A
import os
import sys

In [ ]:
 classlist = ["Correction_fluid","Battery","Cleaning_solution","Light_bulb","phone","Insecticide","Three_eye_plug","Aerosol_cans","Snack","Tissue","Foam_box","Paper_cup","Plastic_box","Drinking_straw","paper","plastic_glass","bottle","can","carton","glass_bottle","news_paper","plastic_bag","plastic_bottle"]

In [16]:
import zipfile
with zipfile.ZipFile("/content/drive/MyDrive/CE_Project/Dataset/forKmodel/Dataset.zip", 'r') as zip_ref:
    zip_ref.extractall("/content/Dataset")

In [17]:
fileNames = [os.path.splitext(filename)[0] for filename in os.listdir("/content/Dataset/labels/")]

In [33]:
x = os.listdir("/content/Dataset/labels/")

In [35]:
len(fileNames)

20954

In [18]:
fileNames.sort()

In [61]:
count = 0
for fileName in fileNames:
  # if not "plastic_glass" in fileName:
  #   continue
  print(fileName)
  if '.ini' in fileName:
    continue
  # for aug in range(5):
  image = cv2.imread("/content/Dataset/images/"+fileName+".jpg")
  image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
  with open("/content/Dataset/labels/"+fileName+".txt") as file:
    bboxes = []
    for line in file:
      # print(line.rstrip())
      bbox = line.rstrip()
      bbox_class = bbox.split(" ")
      # print(bbox_class)
      bbox_class.append(int(bbox_class.pop(0)))
      # bbox_class.pop()
      # print(bbox_class)
      for i in range(len(bbox_class)-1):
        bbox_class[i] = float(bbox_class[i])
        if bbox_class[i] <= 0:
          bbox_class[i] = 0.000001
      # print(bbox_class)
      
      bboxes.append(bbox_class)
      # print("")
      # print("")
      # print(bboxes)

  # if aug == 0:
  transform = A.Compose([
    A.augmentations.geometric.resize.Resize(height=224,width=224,p=1)
  ], bbox_params=A.BboxParams(format='yolo', min_visibility=0.9))
  transformed = transform(image=image, bboxes=bboxes)
  # print(transformed)
  transformed_image = transformed['image']
  
  transformed_bboxes = transformed['bboxes']
  # print(transformed_bboxes)
  # sys.exit()

  # save txt
  result = ''.join([i for i in fileName if not i.isdigit()])

  # for classs in classlist:    
    #print(result+" "+classs)
    # if classs == result:
  # print("==============================================================")
  f= open("/content/Dataset/224/labels/"+fileName+".txt","w+")
  for box_index in range(len(transformed_bboxes)):
    label = list(transformed_bboxes[box_index])
    class_num = label.pop()
    label.insert(0, class_num)
    # print(label)
    aLine = ""
    for value in label:
      if value is int(value):
        aLine = aLine+str(value)
      else:
        if value is 1.0:
          aLine = aLine+"0.999999"
          print("============",temp,"  " ,value ,"========")
        else:
          aLine = aLine + "{:.6f}".format(value)
      aLine = aLine + " "
    f.write(aLine.strip())
    f.write("\n")
  f.close()
  # save img
  cv2.imwrite("/content/Dataset/224/images/"+fileName+".jpg",transformed_image)
  count += 1
  print(count)

Streaming output truncated to the last 5000 lines.
phone240_aug0
18455
phone240_aug1
18456
phone240_aug2
18457
phone241
18458
phone241_aug0
18459
phone241_aug1
18460
phone241_aug2
18461
phone242
18462
phone242_aug0
18463
phone242_aug1
18464
phone242_aug2
18465
phone243
18466
phone243_aug0
18467
phone243_aug1
18468
phone243_aug2
18469
phone244
18470
phone244_aug0
18471
phone244_aug1
18472
phone244_aug2
18473
phone245
18474
phone245_aug0
18475
phone245_aug1
18476
phone245_aug2
18477
phone246
18478
phone246_aug0
18479
phone246_aug1
18480
phone246_aug2
18481
phone247
18482
phone247_aug0
18483
phone247_aug1
18484
phone247_aug2
18485
phone249
18486
phone249_aug0
18487
phone249_aug1
18488
phone249_aug2
18489
phone25
18490
phone250
18491
phone250_aug0
18492
phone250_aug1
18493
phone250_aug2
18494
phone251
18495
phone251_aug0
18496
phone251_aug1
18497
phone251_aug2
18498
phone252
18499
phone252_aug0
18500
phone252_aug1
18501
phone252_aug2
18502
phone253
18503
phone253_aug0
18504
phone253_aug1
1

In [45]:
import os
import shutil

root_src_dir = "/content/Dataset/224"
root_dst_dir = "/content/drive/MyDrive/CE_Project/Dataset/forKmodel/224"

for src_dir, dirs, files in os.walk(root_src_dir):
    dst_dir = src_dir.replace(root_src_dir, root_dst_dir, 1)
    if not os.path.exists(dst_dir):
        # print('making new dir.')
        os.makedirs(dst_dir)
    for file_ in files:
        src_file = os.path.join(src_dir, file_)
        dst_file = os.path.join(dst_dir, file_)
        if os.path.exists(dst_file):
            # print('file already exist.')
            try:
                # print('removing old file.')
                os.remove(dst_file)
            except PermissionError as exc:
                os.chmod(dst_file, stat.S_IWUSR)
                os.remove(dst_file)
        shutil.copy(src_file, dst_dir)
        # print('copy success.')

In [62]:
shutil.make_archive("/content/Dataset/224", 'zip', "/content/Dataset/224")

'/content/Dataset/224.zip'

In [63]:
shutil.copy("/content/Dataset/224.zip", "/content/drive/MyDrive/CE_Project/Dataset/forKmodel/224.zip")

'/content/drive/MyDrive/CE_Project/Dataset/forKmodel/224.zip'

In [64]:
%cd /content/Dataset/images
!ls | wc -l

/content/Dataset/images
20954


In [65]:
%cd /content/Dataset/224/images
!ls | wc -l

/content/Dataset/224/images
20954
